In [12]:
%pip install gensim==4.3.3 --user
%pip uninstall -y nougat-ocr
%pip install transformers==4.45.1
%pip install torch==2.4.1
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.6.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.0-cp312-cp312-macosx_12_0_arm64.whl (11.2 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [72]:
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'
from langchain_community.document_loaders import WebBaseLoader
import bs4
import openai
from langchain import hub
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import PromptTemplate

In [73]:
openai.api_key = os.getenv('OPENAI_API_KEY')
embedding_function = OpenAIEmbeddings()
llm = ChatOpenAI(model_name = "gpt-4o-mini")
user_query = "What are the advantages of using RAG?"

In [74]:
question_embedding = embedding_function.embed_query(user_query)
first_5_numbers = question_embedding[:5]
print(f"User question embedding (first 5 dimensions): {first_5_numbers}")

User question embedding (first 5 dimensions): [-0.006360762752592564, -0.0023306477814912796, 0.015498020686209202, -0.022673549130558968, 0.01783391460776329]


In [75]:
embedding_size = len(question_embedding)
print(f"Embedding size: {embedding_size}")

Embedding size: 1536


In [76]:
loader = WebBaseLoader(
    web_paths=["https://kbourne.github.io/chapter1.html"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header")
                                   )
    )
)
docs = loader.load()

In [77]:
text_splitter = SemanticChunker(embedding_function)
splits = text_splitter.split_documents(docs)



In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_documents = [split.page_content for split in splits]

tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(tfidf_documents)


In [79]:
vocab = tfidf_vectorizer.get_feature_names_out()
tf_values = tfidf_matrix.toarray()
idf_values = tfidf_vectorizer.idf_
word_stats = list(zip(vocab, tf_values.sum(axis=0), idf_values))
word_stats.sort(key = lambda x: x[2], reverse=True)
print("Word\t\tTF\t\tIDF")
print("----\t\t--\t\t---")
for word, tf, idf in word_stats[:10]:
    print(f"{word:<12}\t{tf:.2f}\t\t{idf:.2f}")

Word		TF		IDF
----		--		---
000         	0.16		2.95
1024        	0.04		2.95
123         	0.02		2.95
13          	0.04		2.95
15          	0.01		2.95
16          	0.07		2.95
192         	0.06		2.95
1m          	0.08		2.95
200         	0.08		2.95
2024        	0.01		2.95


In [80]:
tfidf_user_query = [user_query]
new_tfidf_matrix = tfidf_vectorizer.transform(tfidf_user_query)
tfidf_similarity_scores = cosine_similarity(new_tfidf_matrix, tfidf_matrix)

tfidf_top_doc_index = tfidf_similarity_scores.argmax()

print("TF-IDF Top Document:\n", tfidf_documents[tfidf_top_doc_index])

TF-IDF Top Document:
 Can you imagine what you could do with all of the benefits mentioned above, but combined with all of the data within your company, about everything your company has ever done, about your customers and all of their interactions, or about all of your products and services combined with a knowledge of what a specific customer’s needs are? You do not have to imagine it, that is what RAG does! Even smaller companies are not able to access much of their internal data resources very effectively. Larger companies are swimming in petabytes of data that is not readily accessible or is not being fully utilized. Prior to RAG, most of the services you saw that connected customers or employees with the data resources of the company were really just scratching the surface of what is possible compared to if they could access ALL of the data in the company. With the advent of RAG and generative AI in general, corporations are on the precipice of something really, really big. Compa

In [81]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

doc2vec_documents = [split.page_content for split in splits]
doc2vec_tokenized_documents = [doc.lower().split() for doc in doc2vec_documents]

doc2vec_tagged_documents = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(doc2vec_tokenized_documents)]
doc2vec_model = Doc2Vec(doc2vec_tagged_documents, vector_size=100, window=5, min_count=1, workers=4)

doc2vec_model.save("doc2vec_model.bin")

In [82]:
loaded_doc2vec_model = Doc2Vec.load("doc2vec_model.bin")

doc2vec_document_vectors = [loaded_doc2vec_model.dv[str(i)] for i in range(len(doc2vec_documents))]
doc2vec_user_query = [user_query]
doc2vec_tokenized_user_query = [content.lower().split() for content in doc2vec_user_query]
doc2vec_user_query_vector = loaded_doc2vec_model.infer_vector(doc2vec_tokenized_user_query[0])
doc2vec_similarity_scores = cosine_similarity([doc2vec_user_query_vector], doc2vec_document_vectors)
doc2vec_top_doc_index = doc2vec_similarity_scores.argmax()
print("\nDoc2Vec Top Document:\n", doc2vec_documents[doc2vec_top_doc_index])


Doc2Vec Top Document:
 There are also generative models that generate images from text prompts, while others generate video from text prompts. There are other models that generate text descriptions from images. We will talk about these other types of models in Chapter 16, Going Beyond the LLM. But for most of the book, I felt it would keep things simple and let you focus on the core principles of RAG if we focus on the type of model that most RAG pipelines use, the LLM. But I did want to make sure it was clear, that while the book focuses primarily on LLMs, RAG can also be applied to other types of generative models, such as those for images and videos. Some popular examples of LLMs are the OpenAI ChatGPT models, the Meta LLaMA models, Google’s PaLM and Gemini models, and Anthropic’s Claude models. Foundation model
A foundation model is the base model for most LLMs. In the case of ChatGPT, the foundation model is based on the GPT (Generative Pre-trained Transformer) architecture, and 

In [83]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

bert_documents = [split.page_content for split in splits]
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_vector_size = bert_model.config.hidden_size
print(f"Vector size of BERT (base-uncased) embeddings: {bert_vector_size}\n")

bert_tokenized_documents = [bert_tokenizer(doc, return_tensors='pt', max_length=512, truncation=True) for doc in bert_documents]
bert_document_embeddings = []
with torch.no_grad():
    for doc in bert_tokenized_documents:
        bert_outputs = bert_model(**doc)
        bert_doc_embedding = bert_outputs.last_hidden_state[0, 0, :].numpy()
        bert_document_embeddings.append(bert_doc_embedding)

bert_user_query = [user_query]
bert_tokenized_user_query = bert_tokenizer(bert_user_query[0], return_tensors='pt', max_length=512, truncation=True)

bert_user_query_embedding = []
with torch.no_grad():
    bert_outputs = bert_model(**bert_tokenized_user_query)
    bert_user_query_embedding = bert_outputs.last_hidden_state[0, 0, :].numpy()

bert_similarity_scores = cosine_similarity([bert_user_query_embedding], bert_document_embeddings)

bert_top_doc_index = bert_similarity_scores.argmax()

print("BERT Top Document:\n", bert_documents[bert_top_doc_index])


Vector size of BERT (base-uncased) embeddings: 768

BERT Top Document:
 Or if you are developing in a legal field, you may want it to sound more like a lawyer. Vector Store or Vector Database?


In [84]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)
retriever = vectorstore.as_retriever()

In [85]:
result = retriever.invoke(user_query)[0]

print("\nRetrieved Document:\n", result.page_content)


Retrieved Document:
 Can you imagine what you could do with all of the benefits mentioned above, but combined with all of the data within your company, about everything your company has ever done, about your customers and all of their interactions, or about all of your products and services combined with a knowledge of what a specific customer’s needs are? You do not have to imagine it, that is what RAG does! Even smaller companies are not able to access much of their internal data resources very effectively. Larger companies are swimming in petabytes of data that is not readily accessible or is not being fully utilized. Prior to RAG, most of the services you saw that connected customers or employees with the data resources of the company were really just scratching the surface of what is possible compared to if they could access ALL of the data in the company. With the advent of RAG and generative AI in general, corporations are on the precipice of something really, really big. Compa

In [86]:
prompt = hub.pull("jclemens24/rag-prompt")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [87]:
relevance_prompt_template = PromptTemplate.from_template(
    """
    Given the following question and retrieved context, determine if the context is relevant to the question.
    Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
    Return ONLY the numeric score, without any additional text or explanation.

    Question: {question}
    Retrieved Context: {retrieved_context}

    Relevance Score:"""
)

In [89]:
def extract_score(llm_output):
    try:
        score = float(llm_output.strip())
        return score
    except ValueError:
        return 0

def conditional_answer(x):
    relevance_score = extract_score(x['relevance_score'])
    if relevance_score < 4:
        return "I don't know"
    else:
        return x['answer']

In [92]:
# First modify the format_docs to handle Document objects
def format_docs(docs):
    if hasattr(docs, 'page_content'):  # Single Document object
        return docs.page_content
    elif isinstance(docs, list):  # List of Document objects
        return "\n\n".join(doc.page_content for doc in docs)
    elif isinstance(docs, dict):
        return str(docs)
    else:
        return str(docs)

# Modify the chain to properly handle Document objects
rag_chain_from_docs = (
    RunnablePassthrough().assign(
        context = lambda x: format_docs(x["context"])
    )
    | RunnableParallel(
        {
            "relevance_score": (
                RunnablePassthrough()
                | (lambda x: relevance_prompt_template.format(
                    question=str(x['question']),
                    retrieved_context=x['context']  # Already formatted by previous step
                  ))
                | prompt
                | llm
                | StrOutputParser()
            ),
            "answer": (
                RunnablePassthrough()
                | prompt
                | llm
                | StrOutputParser()
            )
        }
    )
    | RunnablePassthrough().assign(final_answer = conditional_answer)
)

rag_chain_with_source = (
    RunnableParallel({
        "context": retriever,
        "question": RunnablePassthrough()
    })
    | RunnablePassthrough().assign(
        answer = rag_chain_from_docs
    )
)

In [94]:
print(type(retriever.invoke(user_query)[0]))

<class 'langchain_core.documents.base.Document'>
